**Ансамбль моделей** — это метод, в котором несколько алгоритмов (или вариации одного и того же) обучаются на одних данных, а итоговый прогноз строится на основе всех полученных от моделей прогнозов.

# <center>Бутстреппинг. Бэггинг</center>

В основе бэггинга лежит статистический метод, который называется **бутстрепом (bootstrap)**. Идея бутстрепа заключается в генерации выборок размера n из исходного датасета размера N путём случайного выбора элементов с повторениями в каждом из наблюдений.

![image.png](https://lms.skillfactory.ru/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block@MATHML_md9_2_1.png)

In [28]:
# Задание 2.2

import numpy as np

a = np.array([4.4, 3.8, 4.8, 4.0, 3.4, 4.2, 5.2])
b = a.mean()
round(np.sum((a - b)**2)/7, 3)

0.317

## <center>Bias и Variance</center>

**Смещение** — это разница между математическим ожиданием для прогноза и реальным значением:

$$Bias[\hat{f}(x)] = E[\hat{f}(x)]-y$$

> Смысл смещения — способность получить лучшую среди всех возможных моделей, то есть максимально точные прогнозы.

**Алгоритмы со стабильно маленьким смещением:**

* KNN (n=1)
* метод опорных векторов
* деревья решений с большой глубиной

**Алгоритмы с большим смещением:**

* линейная регрессия
* логистическая регрессия
* деревья решений с маленькой глубиной

**Разброс** — это величина разницы в результатах обучения модели на разных выборках:

$$\operatorname{Var}[\hat{f}(x)]=\mathrm{E}\left[\left(\mathrm{E}[\hat{f}(x)]-\hat{f}(x)\right)^{2}\right]$$

Разброс характеризует устойчивость модели к изменениям в обучающей выборке:

* Если результат сильно зависит от того, какие объекты присутствуют в выборке, разброс будет большим.

* Если алгоритм работает стабильно вне зависимости от особенностей выборки, разброс будет маленьким.

**Алгоритмы, споказывающие маленький разброс:**

* линейная регрессия
* логистическая регрессия
* деревья решений с маленькой глубиной

**Алгоритмы, споказывающие большой разброс:**

* деревья решений с большой глубиной
* KNN
* метод опорных векторов

> Наша ошибка — это сумма квадрата смещения, разброса и неустранимой случайной ошибки. Теперь мы понимаем, из чего состоит ошибка модели. Такое представление помогает нам исследовать с теоретической точки зрения некоторые алгоритмы машинного обучения и часто используется при изучении ансамблей.

## <center>Бэггинг</center>

Его идея состоит в том, что мы берём несколько независимых моделей и усредняем полученные по ним результаты. Таким образом мы получаем модель, имеющую меньший разброс, так как при её построении мы учли несколько моделей.

Как уже было сказано, в реальности получить много независимых выборок слишком сложно, так как найти столько данных обычно невозможно. Поэтому мы используем бутстреп-выборки.

> Важно отметить, что при бэггинге размер каждой бутстреп-выборки должен совпадать с размером исходной выборки.

* Если мы рассматриваем задачу классификации, то, по сути, модели «голосуют» за свой класс.
* Если мы рассматриваем задачу регрессии, то результат — просто среднее арифметическое прогнозов по всем моделям.

**Резюмируем**:

* Бэггинг даёт уменьшение ошибки в  раз по сравнению с одиночной моделью.
* Бэггинг не уменьшает смещение по сравнению с одиночной моделью.
* Бэггинг уменьшает разброс в  раз по сравнению с одиночной моделью.

> Важно отметить, что эти утверждения выведены и доказаны теоретически и будут выполняться на практике только в том случае, если между ошибками нулевая корреляция.

In [29]:
# Задание 2.7

import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv('data/wineQualityReds.csv')
df.head()

,Unnamed: 0,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [31]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [32]:
df['target'] = np.where(df.quality >=6,1,0)

y= df['target']
X = df.drop(['target', 'quality'], axis= 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
model = LogisticRegression()
model.fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
f1_score(preds_test, y_test)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7390476190476191

In [34]:
model = DecisionTreeClassifier(max_depth = 10, random_state=42)
model.fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
f1_score(preds_test, y_test)

0.7601476014760148

In [35]:
# Задание 2.8

from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(estimator=DecisionTreeClassifier(max_depth = 10, random_state=42), n_estimators=1500, random_state=42).fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
f1_score(preds_test, y_test)

0.8239700374531835

## <center>Случайный лес</center>

![image.pmg](https://lms-cdn.skillfactory.ru/assets/courseware/v1/5447a7937de6636597fac98b4f0a9f62/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md9_3_6.png)

> Одно из важных понятий, которое здесь появляется, — это **метод случайных подпространств**, который используется для построения ансамблей моделей.

Кратко опишем его принцип:

1) Отбираем обучающую выборку.
2) Определяем число моделей, которые войдут в ансамбль.
3) Для каждой модели берём не все признаки, а только часть из них и формируем выборку с использованием случайно выбранного набора признаков.
4) Объединяем все результаты и определяем итоговое решение по объектам.

Алгоритм случайного леса в таком контексте реализуется следующим образом:

1) Для того чтобы построить $i$-е дерево леса, из обучающей выборки $X$ берём случайную подвыборку $X_i$ того же размера, что и вся обучающая выборка.

2) После этого в каждой вершине каждого дерева из $M$ возможных признаков выбираем случайную группу признаков объёма $L$. Для выбранных признаков ищем оптимальное разбиение. Рекомендуется использовать $L=\sqrt{M}$ в задачах классификации и $\frac{M}{3}$ — в задачах регрессии.

3) Для получения предсказания необходимо воспользоваться обычным принципом бэггинга: взять усреднённый ответ в случае регрессии или самый популярный класс — для классификации.

**Ошибка Out-of-Bag** — это способ оценить качество случайного леса.

Для того чтобы найти *out-of-Bag*-оценку:

1) Для каждого объекта $x_i$ получаем предсказания всех деревьев $a_b$, обучавшихся на бутстреп-выборках $X_b$, не содержащих $x_i$.

2) Усредняем эти предсказания.

3) Находим значение ошибки для усреднённого предсказания.

4) Усредняем значение функционала ошибки для всех объектов выборки.

In [36]:
# Задание 3.4

data = pd.read_csv('data/boston (1).csv')
data.head()

,crim_rate,zn,business,river,nit_oxiden,rooms,age,dist,highways_index,tax,pup_per_teaс,lower,target
0,"0,00632",18,"2,31",0,"0,538","6,575","65,2","4,09",1,296,"15,3","4,98",24
1,"0,02731",0,"7,07",0,"0,469","6,421","78,9","4,9671",2,242,"17,8","9,14","21,6"
2,"0,02729",0,"7,07",0,"0,469","7,185","61,1","4,9671",2,242,"17,8","4,03","34,7"
3,"0,03237",0,"2,18",0,"0,458","6,998","45,8","6,0622",3,222,"18,7","2,94","33,4"
4,"0,06905",0,"2,18",0,"0,458","7,147","54,2","6,0622",3,222,"18,7","5,33","36,2"


In [37]:
# # 1.

# import statistics

# X = data.drop('target', axis=1)
# y = data['target']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 13)
# y_train.mean()

In [38]:
# # 2.

# from sklearn.linear_model import LinearRegression 
# from sklearn.metrics import mean_absolute_error as mae

# model_lr = LinearRegression()
# model_lr.fit(X_train, y_train)
# preds_train = model_lr.predict(X_train)
# preds_test = model_lr.predict(X_test)
# print(mae(y_test, preds_test))

In [39]:
# # 3.

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import mean_absolute_error as mae

# model = DecisionTreeRegressor(random_state=13)
# model.fit(X_train, Y_train)
# preds_train = model.predict(X_train)
# preds_test = model.predict(X_test)
# print(mean_absolute_error(y_train, preds_train))
# print(mean_absolute_error(y_test, preds_test))

In [40]:
# # 4.

# from sklearn.ensemble import RandomForestClassifier

# for n in [3, 10, 100, 500]:
#     model = RandomForestRegressor(n_estimators=n, random_state = 13)
#     model.fit(X_train, y_train)
#     preds_train = model.predict(X_train)
#     preds_test = model.predict(X_test)
#     print(mean_absolute_error(y_test, preds_test))

In [41]:
df = pd.read_csv('data/weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [42]:
# Задание 4.1

df.isnull().sum().sum()

343248

In [43]:
# Задание 4.2

round(df.isna().sum() / len(df), 3)

Date             0.000
Location         0.000
MinTemp          0.010
MaxTemp          0.009
Rainfall         0.022
Evaporation      0.432
Sunshine         0.480
WindGustDir      0.071
WindGustSpeed    0.071
WindDir9am       0.073
WindDir3pm       0.029
WindSpeed9am     0.012
WindSpeed3pm     0.021
Humidity9am      0.018
Humidity3pm      0.031
Pressure9am      0.104
Pressure3pm      0.103
Cloud9am         0.384
Cloud3pm         0.408
Temp9am          0.012
Temp3pm          0.025
RainToday        0.022
RainTomorrow     0.022
dtype: float64

In [44]:
df.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1, inplace = True)

In [45]:
# Задание 4.3

df.RainToday = df.RainToday.map({'No': 0, 'Yes': 1})

In [46]:
df.RainTomorrow = df.RainTomorrow.map({'No': 0, 'Yes': 1})

In [47]:
# Задание 4.4

df.Date = pd.to_datetime(df.Date)
df['Month'] = df.Date.dt.month
df.drop('Date', axis = 1, inplace = True)
df_season = df.groupby('Month').mean(numeric_only=True)
df_season[['RainToday']]

,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135


In [48]:
# Задание 4.5

categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df_dummies = pd.get_dummies(df, columns=categoricals)
df_dummies.shape

(145460, 124)

In [49]:
# Задание 4.6

df_dummies.dropna(inplace=True)
X = df_dummies.drop('RainTomorrow', axis = 1)
y = df_dummies['RainTomorrow']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 31)
y_test.mean()

0.22770253002811142

In [50]:
# Задание 4.7

def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inds] #выбираем значения по индексам
    return numbers

In [51]:
target = X_train['MinTemp'].values #выбираем целевую переменную
np.random.seed(31) #задаём параметр генератора случайных чисел
mean_values = [np.mean(x) for x in gbs(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение

0.02879072820657669

In [52]:
# Задание 4.8

from sklearn.metrics import roc_auc_score

clf = LogisticRegression()
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
roc_auc_score(y_test, preds_test)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7265516156243416

In [53]:
# Задание 4.9

params = {
    'max_leaf_nodes': list(range(2, 10)), 
    'min_samples_split': [2, 3, 4], 
    'max_depth': [5,7,9,11]
}

In [54]:
from sklearn.model_selection import GridSearchCV

grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

In [55]:
clf = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

0.7


In [56]:
# Задание 4.10

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100, random_state=31)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

0.73


In [57]:
# Задание 4.11

params = {
    'max_features': [4, 5, 6, 7], 
    'min_samples_leaf': [3, 5, 7, 9, 11], 
    'max_depth': [5, 10, 15]
}

In [58]:
grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=31), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.776 total time=   0.7s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.7s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.776 total time=   0.7s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.7s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.776 total time=   0.7s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   0.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   0.7s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.776 total time=   0.6

In [59]:
clf = RandomForestClassifier(n_estimators = 100, max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

0.7


In [60]:
# Задание 4.12

feature_names = [x for x in df_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': clf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000


# <center>Бустинг</center>